In [1]:
import email
import datetime
from email import policy
from email.parser import BytesParser
import glob
import os
import pandas as pd
from pandas.io.json import json_normalize
import json
from json import JSONEncoder
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


In [3]:
#change filename into foldername_filename

for root, dirs, files in os.walk('/home/jupyter/maildir'):
    if not files:
        continue
    prefix = os.path.basename(root)
    for f in files:
        os.rename(os.path.join(root, f), os.path.join(root, "{}_{}".format(prefix, f)))

In [4]:
os.chdir('maildir/sent_mail')

In [5]:
import os, re
#!pip install eml_parser[file-magic]

data_full=[]

import datetime
import json
import eml_parser
from pandas.io.json import json_normalize

def json_serial(obj):
    if isinstance(obj, datetime.datetime):
        serial = obj.isoformat()
        return serial


file_list = glob.glob('*.eml') # returns list of files
for file in file_list:
    with open(file,'rb') as fp:
            #raw_email = fp.read()
        msg = BytesParser(policy=policy.default).parse(fp)
        raw_email = fp.read()
        parsed_eml = eml_parser.eml_parser.decode_email_b(raw_email)
        fnm = os.path.splitext(file)[0]
        sent_to= msg['to']
        sent_from= msg['from']
            #cleaning the data 

        txt = msg.get_content()
            # Replace any whitespace (newline, tabs, etc.) by a single space
        txt = re.sub('\\s', ' ', txt)

        json.dumps(txt, default=json_serial)
        json.dumps(sent_to, default=json_serial)
        json.dumps(sent_from, default=json_serial)
            #data_full.append(txt)

            #data_full=pd.DataFrame(columns=['To','Message'])
        data_full.append([fnm,sent_from,sent_to,txt])


sent_mail = pd.DataFrame(data_full)
            #data_full.append(data_df),
sent_mail.columns =['Email_Num','From','To', 'Message']
sent_mail['Message']=sent_mail['Message'].str.replace('\\n',' ').str.replace('-','').str.replace('?','').str.replace('(','').str.replace(')','').str.replace('/','').str.replace('~','').str.replace('#','')

sent_mail.sort_values('Email_Num', ascending= True)

print(sent_mail)
        

                                       Email_Num                     From  \
0    sent_mail_sent_mail_sent_mail_sent_mail_420  phillip.allen@enron.com   
1    sent_mail_sent_mail_sent_mail_sent_mail_455  phillip.allen@enron.com   
2    sent_mail_sent_mail_sent_mail_sent_mail_440  phillip.allen@enron.com   
3    sent_mail_sent_mail_sent_mail_sent_mail_175  phillip.allen@enron.com   
4    sent_mail_sent_mail_sent_mail_sent_mail_318  phillip.allen@enron.com   
..                                           ...                      ...   
597  sent_mail_sent_mail_sent_mail_sent_mail_216  phillip.allen@enron.com   
598  sent_mail_sent_mail_sent_mail_sent_mail_225  phillip.allen@enron.com   
599  sent_mail_sent_mail_sent_mail_sent_mail_512  phillip.allen@enron.com   
600  sent_mail_sent_mail_sent_mail_sent_mail_435  phillip.allen@enron.com   
601  sent_mail_sent_mail_sent_mail_sent_mail_250  phillip.allen@enron.com   

                                                    To  \
0                

In [6]:
df=sent_mail
df

,Email_Num,From,To,Message
0,sent_mail_sent_mail_sent_mail_sent_mail_420,phillip.allen@enron.com,barry.tycholiz@enron.com,no problem
1,sent_mail_sent_mail_sent_mail_sent_mail_455,phillip.allen@enron.com,tori.kuykendall@enron.com,Forwarded by Phillip K AllenHOUECT on 0313200...
2,sent_mail_sent_mail_sent_mail_sent_mail_440,phillip.allen@enron.com,llewter@austin.rr.com,"Larrry, I realize you are disappointed about ..."
3,sent_mail_sent_mail_sent_mail_sent_mail_175,phillip.allen@enron.com,stagecoachmama@hotmail.com,"Lucy, I got your email with the attachment. ..."
4,sent_mail_sent_mail_sent_mail_sent_mail_318,phillip.allen@enron.com,keith.holst@enron.com,Forwarded by Phillip K AllenHOUECT on 0211200...
...,...,...,...,...
597,sent_mail_sent_mail_sent_mail_sent_mail_216,phillip.allen@enron.com,keith.holst@enron.com,Forwarded by Phillip K AllenHOUECT on 0713200...
598,sent_mail_sent_mail_sent_mail_sent_mail_225,phillip.allen@enron.com,"matthew.lenhart@enron.com, kenneth.shulklapper...",Forwarded by Phillip K AllenHOUECT on 0710200...
599,sent_mail_sent_mail_sent_mail_sent_mail_512,phillip.allen@enron.com,mark.whitt@enron.com,Forwarded by Phillip K AllenHOUECT on 0212200...
600,sent_mail_sent_mail_sent_mail_sent_mail_435,phillip.allen@enron.com,llewter@austin.rr.com,"Larry, Jacques has been working with Claudia...."


In [7]:
#split "To" emails into different rows
df['To'] = df['To'].str.split(',')
df = (df
 .set_index(['Email_Num','From','Message'])['To']
 .apply(pd.Series)
 .stack()
 .reset_index()
 .drop('level_3', axis=1)
 .rename(columns={0:'To'}))


In [8]:
df

,Email_Num,From,Message,To
0,sent_mail_sent_mail_sent_mail_sent_mail_420,phillip.allen@enron.com,no problem,barry.tycholiz@enron.com
1,sent_mail_sent_mail_sent_mail_sent_mail_455,phillip.allen@enron.com,Forwarded by Phillip K AllenHOUECT on 0313200...,tori.kuykendall@enron.com
2,sent_mail_sent_mail_sent_mail_sent_mail_440,phillip.allen@enron.com,"Larrry, I realize you are disappointed about ...",llewter@austin.rr.com
3,sent_mail_sent_mail_sent_mail_sent_mail_175,phillip.allen@enron.com,"Lucy, I got your email with the attachment. ...",stagecoachmama@hotmail.com
4,sent_mail_sent_mail_sent_mail_sent_mail_318,phillip.allen@enron.com,Forwarded by Phillip K AllenHOUECT on 0211200...,keith.holst@enron.com
...,...,...,...,...
713,sent_mail_sent_mail_sent_mail_sent_mail_225,phillip.allen@enron.com,Forwarded by Phillip K AllenHOUECT on 0710200...,matthew.lenhart@enron.com
714,sent_mail_sent_mail_sent_mail_sent_mail_225,phillip.allen@enron.com,Forwarded by Phillip K AllenHOUECT on 0710200...,kenneth.shulklapper@enron.com
715,sent_mail_sent_mail_sent_mail_sent_mail_512,phillip.allen@enron.com,Forwarded by Phillip K AllenHOUECT on 0212200...,mark.whitt@enron.com
716,sent_mail_sent_mail_sent_mail_sent_mail_435,phillip.allen@enron.com,"Larry, Jacques has been working with Claudia....",llewter@austin.rr.com


In [9]:
#cleaning forwaded email
df['Message'] = df['Message'].str.rsplit('Subject: ').str[-1] 
df['Message'] = df['Message'].str.rsplit(' --------------------------- ').str[-1]


In [13]:
df.groupby(['To'])["Email_Num"].count()

To
 brent.price@enron.com        1
 djack@keyad.com              1
 dmccarty@enron.com           2
 frank.ermis@enron.com       17
 hunter.shively@enron.com     2
                             ..
torrey.moorer@enron.com       1
vladimir.gorny@enron.com      2
william.kelly@enron.com       4
ywang@enron.com               2
zimam@enron.com               1
Name: Email_Num, Length: 218, dtype: int64

In [ ]:
#creating and cleaning author2doc part\n",

author2doc = dict()
i=0
  
eml_ids=[],
#eml_ids.append(df['Email_Num'])

# iterate through each row and select
# 'Email_Num' column
for ind in df.index: 
    eml_ids= df['Email_Num'][ind]

    sent_to_name=re.sub('@enron.com', '', str(df['To'][ind]))

    if not author2doc.get(sent_to_name):
            # This is a new person
        author2doc[sent_to_name] = []
        author2doc[sent_to_name].extend(([df['Email_Num'][ind]]))

        i += 1
# Mapping from ID of email to an integer ID.
eml_ids_dict= dict(zip(df['Email_Num'], range(len(df['Email_Num']))))

In [ ]:
#mapping email sent_to integer ID
for a, a_eml_ids in author2doc.items():
    for i, eml_id in enumerate(a_eml_ids):
        author2doc[a][i] = eml_ids_dict[eml_id]

Pre-processing text

The text will be pre-processed using the following steps:

    Tokenize text
    Replace all whitespace by single spaces
    Remove all punctuation and numbers
    Remove stopwords
    Lemmatize words
    Add multi-word named entities
    Add frequent bigrams
    Remove frequent and rare words

In [ ]:
import spacy
!python3 -m spacy download en
nlp = spacy.load('en')

In [ ]:
%%time

docs=[]

for ind in df.index: 
    eml_message= df['Message'][ind]
    docs.append(eml_message)

processed_docs = []    
for doc in nlp.pipe(docs, n_threads=4, batch_size=100):
    # Process document using Spacy NLP pipeline.
    
    ents = doc.ents  # Named entities.

    # Keep only words (no numbers, no punctuation)
    # Lemmatize tokens, remove punctuation and remove stopwords
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Remove common words from a stopword list
    #doc = [token for token in doc if token not in STOPWORDS]

    # Add named entities, but only if they are a compound of more than word
    doc.extend([str(entity) for entity in ents if len(entity) > 1])

    processed_docs.append(doc)

In [ ]:
docs = processed_docs
del processed_docs

In [ ]:
*****

In [ ]:
df.Message.head(20)

In [ ]:
#Prepare stop word
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['re' ])

In [ ]:
#Remove emails and newline characters

# Convert to list
data = df.Message.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

In [ ]:
#Tokenize words and Clean-up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

In [ ]:
#Creating Bigram and Trigram Models

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

In [ ]:
#Remove Stopwords, Make Bigrams and Lemmatize
# Define functions for stopwords, bigrams, trigrams and lemmatization

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
#calling the functions

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
#train and use model
from gensim.models import AuthorTopicModel
%time model = AuthorTopicModel(corpus=corpus, num_topics=10, id2word=id2word, \
                author2doc=author2doc, chunksize=2000, passes=1, eval_every=0, \
                iterations=1, random_state=1)

In [ ]:
%%time
model_list = []
for i in range(5):
    model = AuthorTopicModel(corpus=corpus, num_topics=10, id2word=id2word, \
                    author2doc=author2doc, chunksize=2000, passes=100, gamma_threshold=1e-10, \
                    eval_every=0, iterations=1, random_state=i)
    top_topics = model.top_topics(corpus)
    tc = sum([t[1] for t in top_topics])
    model_list.append((model, tc))

In [ ]:
model, tc = max(model_list, key=lambda x: x[1])
print('Topic coherence: %.3e' %tc)

In [ ]:
model.show_topic(3)

In [ ]:
#looking at different topics

topic_labels = ['1', '2', '3', '4', \
               '5', '6', '7', \
                '8', '9', '10']


for topic in model.show_topics(num_topics=10):
    print('Label: ' + topic_labels[topic[0]])
    words = ''
    for word, prob in model.show_topic(topic[0]):
        words += word + ' '
    print('Words: ' + words)
    print()

In [ ]:
#Let's print the top topics of some authors. First, we make a function to help us do this more easily


def show_author(name):
    print('\n%s' % name)
    print('Docs:', model.author2doc[name])
    print('Topics:')
    pprint([(topic_labels[topic[0]], topic[1]) for topic in model[name]])

In [ ]:
(lambda x: set(x[:5].index))(df['To'].value_counts())

In [ ]:
show_author('jsmith@austintx.com')